<a href="https://colab.research.google.com/github/kunalnischal7/NLP_Projects/blob/main/MoveWiseAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoveWiseAI

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
import google.generativeai as genai
from google.colab import drive

### Connecting with Google Gemini

In [ ]:
genai.configure(api_key="Your_Google_Gemini_API_Key")

## Data Preparation

### Load Data

In [ ]:
drive.mount('/content/drive')
CSV_FILE = '/content/drive/MyDrive/logistics_data_updated.csv'
print("Loading dataset...")
df = pd.read_csv(CSV_FILE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading dataset...


In [ ]:
df

,Article_ID,Facility,Start_Timestamp,End_Timestamp,Processing_Time_Min,Day,Weight,Customer_ID
0,1,Perth,2025-10-12 02:51:46,2025-10-12 06:36:46,57,Sunday,26.23,CUST4252
1,1,Adelaide,2025-10-12 06:36:46,2025-10-12 13:25:46,118,Sunday,26.23,CUST4252
2,1,Adelaide,2025-10-12 13:25:46,2025-10-13 03:47:46,140,Sunday,26.23,CUST4252
3,1,Sydney,2025-10-13 03:47:46,2025-10-13 15:38:46,142,Monday,26.23,CUST4252
4,1,Sydney,2025-10-13 15:38:46,2025-10-14 05:10:46,151,Monday,26.23,CUST4252
...,...,...,...,...,...,...,...,...
4459,999,Canberra,2025-04-12 19:43:54,2025-04-13 02:42:54,67,Saturday,22.98,CUST3781
4460,1000,Perth,2025-07-17 07:36:33,2025-07-17 13:09:33,60,Thursday,13.63,CUST2617
4461,1000,Melbourne,2025-07-17 13:09:33,2025-07-17 22:41:33,149,Thursday,13.63,CUST2617
4462,1000,Brisbane,2025-07-17 22:41:33,2025-07-18 17:03:33,77,Thursday,13.63,CUST2617


### Ensuring data has all the columns

In [ ]:
required_columns = {"Article_ID", "Facility", "Start_Timestamp", "End_Timestamp", "Processing_Time_Min", "Day", "Weight", "Customer_ID"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing columns. Found: {df.columns}")

### Convert timestamps to datetime

In [ ]:
df['Start_Timestamp'] = pd.to_datetime(df['Start_Timestamp'])
df['End_Timestamp'] = pd.to_datetime(df['End_Timestamp'])

### Extracting Date Parts for Trend Analysis

In [ ]:
df['Day'] = df['Start_Timestamp'].dt.day_name()
df['Month'] = df['Start_Timestamp'].dt.month_name()

df.head()

## SQL

In [ ]:
# Initialize SQLAlchemy Engine
engine = create_engine('sqlite:///:memory:')  # Using an in-memory SQLite database

# Create the SQL table
df.to_sql('articles', con=engine, if_exists='replace', index=False)

4464

### SQL Generation Function

In [ ]:
def generate_sql(natural_query):
    prompt = f"""
    **Task**: Convert natural language to precise SQL
    **Schema**:
    - articles (
        Article_ID INT,
        Facility TEXT,
        Start_Timestamp DATETIME,
        End_Timestamp DATETIME,
        Processing_Time_Min INT,
        Day TEXT,
        Weight FLOAT,
        Customer_ID TEXT
    )

    **Rules**:
    1. Use EXACT facility names: {sorted(df['Facility'].unique())}
    2. For counts, use COUNT(DISTINCT Article_ID)
    3. Use GROUP BY for facility/customer/day/month aggregations
    4. Use ORDER BY and LIMIT for "maximum" or "highest" queries
    5. For time calculations, use Processing_Time_Min column
    6. For trends, consider Day and Month columns
    7. For customer analysis, use Customer_ID field

    **Examples**:
    - "Facility with most articles" →
      SELECT Facility, COUNT(DISTINCT Article_ID) AS Article_Count
      FROM articles
      GROUP BY Facility
      ORDER BY Article_Count DESC
      LIMIT 1

    - "Average processing time per facility" →
      SELECT Facility, AVG(Processing_Time_Min) AS Avg_Processing_Time
      FROM articles
      GROUP BY Facility

    - "Customer with most shipments" →
      SELECT Customer_ID, COUNT(DISTINCT Article_ID) AS Shipment_Count
      FROM articles
      GROUP BY Customer_ID
      ORDER BY Shipment_Count DESC
      LIMIT 1

    - "Daily average processing time" →
      SELECT Day, AVG(Processing_Time_Min) AS Avg_Processing_Time
      FROM articles
      GROUP BY Day

    - "Monthly shipment count" →
      SELECT Month, COUNT(DISTINCT Article_ID) AS Shipment_Count
      FROM articles
      GROUP BY Month

    **Query to Convert**: "{natural_query}"
    """

    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    return response.text.strip().replace('```sql', '').replace('```', '').strip()

### Validation Function

In [ ]:
def validate_sql(sql):
    required_terms = {'SELECT', 'FROM', 'ARTICLES'}
    if 'GROUP BY' in sql.upper() and ('COUNT(' not in sql.upper() and 'AVG(' not in sql.upper()):
        raise ValueError("Aggregation queries must include COUNT/AVG")
    return all(term in sql.upper() for term in required_terms)

### Execute SQL

In [ ]:
def execute_sql(sql_query):
    try:
        # Validate SQL query for safety
        if not sql_query.strip().upper().startswith("SELECT"):
            raise ValueError("Only SELECT queries are allowed for security reasons.")

        # Check for common SQL injection patterns
        if any(keyword in sql_query.upper() for keyword in ["DROP", "DELETE", "UPDATE", "INSERT", "ALTER"]):
            raise ValueError("Modification queries are not permitted.")

        # Execute the query
        with engine.connect() as conn:
            result = conn.execute(text(sql_query))
            rows = result.fetchall()

            # Handle empty results
            if not rows:
                return None  # Return None for no results

            # Always return rows as lists of tuples
            return rows

    except Exception as e:
        return f"Error executing SQL: {str(e)}"

### Explanation Function

In [ ]:
def explain_results(results, original_query, sql):
    explanation = ""

    # Handle numerical results
    if any(kw in sql.upper() for kw in ['AVG', 'COUNT', 'SUM']):
        if len(results[0]) == 2:  # Grouped results
            items = [f"{row[0]}: {row[1]:.2f}" if isinstance(row[1], float) else f"{row[0]}: {row[1]}"
                    for row in results]
            explanation = "\n".join(items)
        else:  # Single value
            explanation = f"Result: {results[0][0]:.2f}" if isinstance(results[0][0], float) else f"Result: {results[0][0]}"

    # Handle time-based trends
    elif 'DAY' in sql.upper() or 'MONTH' in sql.upper():
        explanation = "Trends:\n" + "\n".join([f"{row[0]}: {row[1]}" for row in results])

    # Handle timestamp queries
    elif 'START_TIMESTAMP' in sql.upper() or 'END_TIMESTAMP' in sql.upper():
        # Format timestamps to remove extra zeros
        timestamps = [pd.to_datetime(row[0]).strftime('%Y-%m-%d %H:%M:%S') for row in results]
        explanation = "Start Timestamps:\n" + "\n".join(timestamps)

    else:
        if len(results[0]) == 1:  # Single column
            explanation = "Results:\n" + "\n".join([str(row[0]) for row in results])
        else:
            explanation = "Results:\n" + "\n".join([str(row) for row in results])

    return explanation

## Main Interaction Loop

In [ ]:
# Main interaction loop
while True:
    user_question = input("\nAsk a question (or 'exit'): ").strip()
    if user_question.lower() == 'exit':
        break

    # Generate SQL with validation
    try:
        sql = generate_sql(user_question)
        if not validate_sql(sql):
            raise ValueError("Generated SQL failed validation checks")

        print(f"\nGenerated SQL: {sql}")

        # Execute SQL
        results = execute_sql(sql)
        if results is None:  # No results found
            print("\n🔍 No matching records found in the dataset.")
        elif isinstance(results, str):  # Error occurred
            print(f"\n❌ {results}")
        else:
            # Explain results
            explanation = explain_results(results, user_question, sql)
            print(f"\n🔍 Analysis: {explanation}")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("Tip: Try rephrasing with facility names from: " + ", ".join(df['Facility'].unique()))


Generated SQL: SELECT
  Start_Timestamp
FROM articles
WHERE
  Article_ID = 214;

🔍 Analysis: Start Timestamps:
2025-07-13 05:44:50
2025-07-13 10:11:50
2025-07-14 01:22:50
2025-07-14 08:07:50

Generated SQL: SELECT Facility, AVG(Processing_Time_Min) AS Avg_Processing_Time
FROM articles
GROUP BY Facility
ORDER BY Avg_Processing_Time DESC
LIMIT 1;

🔍 Analysis: Geelong: 108.21

Generated SQL: SELECT
  Month,
  COUNT(DISTINCT Article_ID) AS Shipment_Count
FROM articles
GROUP BY
  Month;

🔍 Analysis: April: 97
August: 67
December: 86
February: 78
January: 83
July: 80
June: 82
March: 94
May: 99
November: 89
October: 79
September: 84

Generated SQL: SELECT Customer_ID, COUNT(DISTINCT Article_ID) AS Shipment_Count
FROM articles
GROUP BY Customer_ID
ORDER BY Shipment_Count DESC
LIMIT 1;

🔍 Analysis: CUST3548: 17

Generated SQL: SELECT Facility, AVG(Weight) AS Avg_Package_Weight
FROM articles
GROUP BY Facility;

🔍 Analysis: Adelaide: 24.46
Brisbane: 24.56
Canberra: 24.46
Geelong: 23.61
Gold Coas